In [1]:
from statsbombpy import sb
import pandas as pd
import numpy as np
import random
import json
from openai import OpenAI

In [55]:
# Replace random selector with list of specific indices - matched by scoreline
#free_comps = sb.competitions()
wsl_20_21 = sb.matches(competition_id = 37, season_id = 90).sort_values(by = "match_date").reset_index(drop = True)
indices = [13] # selected games to match scorelines of randomly selected matches in Method 1
#for i in range(0,5):
   # indices.append(random.randint(0,131))
#print(indices)


C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [56]:
#wsl_20_21[(wsl_20_21["home_score"] == 3) & (wsl_20_21["away_score"] == 0)]

In [57]:
# Select a match ID to generate a report
selected_matches = wsl_20_21.iloc[indices].reset_index()
selected_matches

,index,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,13,3775562,2020-10-04,13:00:00.000,England - FA Women's Super League,2020/2021,Manchester United,Brighton & Hove Albion WFC,3,0,...,2021-06-13T16:17:31.694,3,Regular Season,Leigh Sports Village Stadium,Elliot Swallow,Casey Stoney,Hope Patricia Powell,1.1.0,2,2


In [58]:
# Compute the summary statistics for the match 
# Calculate possession for each team
def calculate_possession(match_df):
    home_team = match_df["team"][0]
    away_team = match_df["team"][1]
    home_team_pos = 0
    away_team_pos = 0
    for index, row in match_df.iterrows():
        if row["possession_team"] == home_team:
            home_team_pos +=1
        elif row["possession_team"] == away_team:
            away_team_pos +=1
        else:
            continue
    return [round(home_team_pos/(home_team_pos + away_team_pos),2)*100, round(away_team_pos/(home_team_pos + away_team_pos),2)*100]
    
# Calculate in game statistics
match_data_dict = {}
touch_events = ['Pass', 'Dribble', 'Shot', 'Interception', 'Clearance', 'Ball Recovery']
def match_data_extract(match_df, dictionary):
        for index, row in match_df.iterrows():
        # Store matchday information for each match_id in the dictionary
            dictionary[f"{row["match_date"]}:{row["home_team"]} vs. {row["away_team"]}"] = {}
            match_id = row["match_id"]
            match_date = row["match_date"]
            match_week = row["match_week"]
            stadium = row["stadium"]
            referee = row["referee"]
            home_manager = row["home_managers"]
            away_manager = row["away_managers"]
            home_team = row["home_team"]
            away_team = row["away_team"]
            home_score = row["home_score"]
            away_score = row["away_score"]
            match_result = "Home Win" if home_score > away_score else "Away Win" if away_score > home_score else "Draw"
            winning_team = home_team if match_result == "Home Win" else away_team if match_result == "Away Win" else "Draw"
            game = sb.events(match_id = match_id)
            game["minute"] = game["minute"] + 1 # Convert to ordered time i.e: minute = 80 - event occurred in 81st minute
            game[["location_x", "location_y"]] = game["location"].apply(pd.Series)
            game[["pass_end_location_x", "pass_end_location_y"]] = game["pass_end_location"].apply(pd.Series)
            game["pass_distance"] = round(np.sqrt((game["pass_end_location_x"] - game["location_x"])**2 + (game["pass_end_location_y"]- game["location_y"])**2))
            # Extract count based stats, home, away, and total

            # Half-time score
            first_half_home = game.loc[(game["team"] == home_team) & (game["period"] == 1)]
            first_half_away = game.loc[(game["team"] == away_team) & (game["period"] == 1)]

            half_time_score_home = first_half_home.loc[(first_half_home["type"] == "Shot") & (first_half_home["shot_outcome"] == "Goal")].shape[0]
            half_time_score_away = first_half_away.loc[(first_half_away["type"] == "Shot") & (first_half_away["shot_outcome"] == "Goal")].shape[0]
            
            # Clearances
            # Unusually large values for clearances
            # Count adjusted to include only 1 clearance for each unique possession
            # Goal kicks and free kicks not included in clearance count            
            home_clearances = game.loc[(game["team"] == home_team) & (game["type"] == "Clearance")].drop_duplicates('possession')
            home_clearances = home_clearances[~home_clearances["play_pattern"].isin(['From Goal Kick', 'From Free Kick'])].shape[0]
            away_clearances = game.loc[(game["team"] == away_team) & (game["type"] == "Clearance")].drop_duplicates('possession')
            away_clearances = away_clearances[~away_clearances["play_pattern"].isin(['From Goal Kick', 'From Free Kick'])].shape[0]
            total_clearances = home_clearances + away_clearances
            
            # Shots
            home_shots = game.loc[(game["team"] == home_team) & (game["type"] == "Shot")].shape[0]
            away_shots = game.loc[(game["team"] == away_team) & (game["type"] == "Shot")].shape[0]
            total_shots = home_shots + away_shots
            
            # Shots on Target
            home_SoT = game.loc[(game["team"] == home_team) & (game["type"] == "Shot")]
            home_SoT = home_SoT[~home_SoT["shot_outcome"].isin(["Blocked", "Wayward", "Off T"])].shape[0]
            away_SoT = game.loc[(game["team"] == away_team) & (game["type"] == "Shot")]
            away_SoT = away_SoT[~away_SoT["shot_outcome"].isin(["Blocked", "Wayward", "Off T"])].shape[0]
            total_SoT = home_SoT + away_SoT

            # Goal times
            # Identify goals for each team and extract a list of the minute values
            home_goal_times = game.loc[(game["team"] == home_team) & (game["shot_outcome"] == "Goal")]["minute"].tolist()
            away_goal_times = game.loc[(game["team"] == away_team) & (game["shot_outcome"] == "Goal")]["minute"].tolist()
            all_goal_times = home_goal_times + away_goal_times
            
            # Possession %
            home_possession = calculate_possession(game)[0]
            away_possession = calculate_possession(game)[1]
            
            # Corners
            home_corners = game.loc[(game["team"] == home_team) & (game["pass_type"] == "Corner")].shape[0]
            away_corners = game.loc[(game["team"] == away_team) & (game["pass_type"] == "Corner")].shape[0]
            total_corners = home_corners + away_corners
            
            #Fouls Conceded
            home_fouls_committed = game.loc[(game["team"] == home_team) & (game["type"] == "Foul Committed")].shape[0]
            away_fouls_committed = game.loc[(game["team"] == away_team) & (game["type"] == "Foul Committed")].shape[0]
            total_fouls = home_fouls_committed + away_fouls_committed
            
            #Offsides
            home_offside = game.loc[(game["team"] == home_team) & (game["type"] == "Offside")].shape[0]
            away_offside = game.loc[(game["team"] == away_team) & (game["type"] == "Offside")].shape[0]
            total_offside = home_offside + away_offside
            
            #Passes
            home_passes = game.loc[(game["team"] == home_team) & (game["type"] == "Pass") & (game["pass_outcome"].isnull())]
            away_passes = game.loc[(game["team"] == away_team) & (game["type"] == "Pass") & (game["pass_outcome"].isnull())]

            home_passes_count = home_passes.shape[0]
            away_passes_count = away_passes.shape[0]
            total_passes_count = home_passes_count + away_passes_count

            # Passes into 18 yard box (18 - 62 y)
            home_passes_into_box = home_passes.loc[(home_passes["pass_end_location_x"] > 102) & ((home_passes["pass_end_location_y"] > 18) & (home_passes["pass_end_location_y"] < 62))].shape[0]
            away_passes_into_box = away_passes.loc[(away_passes["pass_end_location_x"] > 102) & ((home_passes["pass_end_location_y"] > 18) & (home_passes["pass_end_location_y"] < 62))].shape[0]
            total_passes_into_box = home_passes_into_box + away_passes_into_box

            # Calculate subsets of pass length 
            home_passes_short = home_passes[home_passes["pass_distance"] < 15].shape[0]
            home_passes_medium = home_passes[(home_passes["pass_distance"] < 30) & (home_passes["pass_distance"] >15)].shape[0] 
            home_passes_long = home_passes[home_passes["pass_distance"] > 30].shape[0]

            away_passes_short = away_passes[away_passes["pass_distance"] < 15].shape[0]
            away_passes_medium = away_passes[(away_passes["pass_distance"] < 30) & (away_passes["pass_distance"] >15)].shape[0]
            away_passes_long = away_passes[away_passes["pass_distance"] > 30].shape[0]

            home_average_pass_length = round(np.mean(home_passes["pass_distance"]),2)
            away_average_pass_length = round(np.mean(away_passes["pass_distance"]),2)
            
            #Tackles
            home_tackles = game.loc[(game["team"] == home_team) & (game["duel_type"] == "Tackle") & (game["duel_outcome"] == "Won")].shape[0]
            away_tackles = game.loc[(game["team"] == away_team) & (game["duel_type"] == "Tackle") & (game["duel_outcome"] == "Won")].shape[0]
            total_tackles = home_tackles + away_tackles
            
            #Touches
            home_touches = game.loc[(game["team"] == home_team) & (game["type"].isin(touch_events))].drop_duplicates("id").shape[0]
            away_touches = game.loc[(game["team"] == away_team) & (game["type"].isin(touch_events))].drop_duplicates("id").shape[0]
            total_touches = home_touches + away_touches
            
            # Yellow and Red Cards

            # Identify rows/ columns where a Yellow/ Red card is listed
            yellows = game[game.apply(lambda col: col.astype(str).str.contains('Yellow Card')).any(axis=1)] # https://stackoverflow.com/a/78718132
            second_yellows = game[game.apply(lambda col: col.astype(str).str.contains('Second Yellow')).any(axis=1)]
            reds = game[(game.apply(lambda col: col.astype(str).str.contains('Red Card')).any(axis=1)) | \
            (game.apply(lambda col: col.astype(str).str.contains('Second Yellow')).any(axis=1))]

            # Combine all yellow card rows for home_team
            home_yellows = yellows.loc[yellows["team"] == home_team] + second_yellows.loc[second_yellows["team"] == home_team]
            # Combine all yellow card rows for away_team
            away_yellows = yellows.loc[yellows["team"] == away_team] + second_yellows.loc[second_yellows["team"] == away_team]
            
            # Subset red card rows for each team
            home_reds = reds.loc[reds["team"] == home_team] 
            away_reds = reds.loc[reds["team"] == away_team]

            # Extract row counts for each card type per team
            home_yellows_int = home_yellows.shape[0]
            away_yellows_int = away_yellows.shape[0]
            home_reds_int = home_reds.shape[0] 
            away_reds_int = away_reds.shape[0]
    
            total_yellows = home_yellows_int + away_yellows_int
            total_reds = home_reds_int + away_reds_int

            # Extract a list of times for each yellow card for the home_team
            home_first_yellow_times = yellows.loc[yellows["team"] == home_team]["minute"].tolist()
            home_second_yellow_times = second_yellows.loc[second_yellows["team"] == home_team]["minute"].tolist()
            home_yellow_times = home_first_yellow_times + home_second_yellow_times

            # Extract a list of times for each yellow card for the away_team
            away_first_yellow_times = yellows.loc[yellows["team"] == away_team]["minute"].tolist()
            away_second_yellow_times = second_yellows.loc[second_yellows["team"] == away_team]["minute"].tolist()
            away_yellow_times = away_first_yellow_times + away_second_yellow_times

            # Extract lists of the times when a red card was issued
            home_red_times = home_reds["minute"].tolist()
            away_red_times = away_reds["minute"].tolist()

            # Combine the lists from each team for all cards
            all_yellow_times = home_yellow_times + away_yellow_times
            all_red_times = home_red_times + away_red_times

            # Full time score
            full_time_score = f"{home_score}: {away_score}"
            
            # Build a nested dictionary of all games
            dictionary[f"{row["match_date"]}:{row["home_team"]} vs. {row["away_team"]}"].update({
                "match_id": match_id,
                "match_date": match_date,
                "match_result": match_result,
                "full_time_score" : full_time_score,
                "match_week" : match_week,
                "stadium" : stadium,
                "referee" : referee,
                "home_manager" : home_manager,
                "away_manager" : away_manager,
                "full_time_score" : full_time_score,
                "home_team": home_team,
                "away_team": away_team,
                "winning team": winning_team,
                "half_time_score_home": half_time_score_home,
                "half_time_score_away": half_time_score_away,
                "home_goal_times": home_goal_times,
                "away_goal_times": away_goal_times,
                "home_clearances": home_clearances,
                "away_clearances": away_clearances,
                "home_shots": home_shots,
                "away_shots": away_shots,
                "home_shot_on_target": home_SoT,
                "away_shot_on_target": away_SoT,
                "home_possession": home_possession,
                "away_possession": away_possession,
                "home_corners": home_corners,
                "away_corners": away_corners, 
                "home_fouls_committed": home_fouls_committed,
                "away_fouls_committed": away_fouls_committed,
                "home_offsides": home_offside,
                "away_offside": away_offside,
                "home_passes": home_passes_count,
                "away_passes": away_passes_count,
                "home_passes_into_box": home_passes_into_box,
                "away_passes_into_box": away_passes_into_box,
                "home_passes_short": home_passes_short,
                "home_passes_medium": home_passes_medium, 
                "home_passes_long": home_passes_long, 
                "away_passes_short": away_passes_short, 
                "away_passes_medium": away_passes_medium, 
                "away_passes_long": away_passes_long,
                "home_average_pass_length": home_average_pass_length,
                "away_average_pass_length": away_average_pass_length,
                "home_tackles": home_tackles,
                "away_tackles": away_tackles,
                "home_touches": home_touches,
                "away_touches": away_touches,
                "home_yellow_cards": home_yellows_int,
                "away_yellow_cards": away_yellows_int,
                "home_red_cards": home_reds_int,
                "away_red_cards": away_reds_int,
                "home_yellow_times": home_yellow_times,
                "away_yellow_times": away_yellow_times,
                "home_red_times": home_red_times,
                "away_red_times": away_red_times,
            })


In [59]:
# Extract event highlights for the match
match_events_dict = {}
def match_events_extract(match_df, dictionary):
        for index, row in match_df.iterrows():
        # Store matchday information for each match_id in the dictionary
            match_id = row["match_id"]
            match_date = row["match_date"]
            home_team = row["home_team"]
            away_team = row["away_team"]
            dictionary[f"{match_date}: {home_team} vs. {away_team}"] = {}
            game = sb.events(match_id = match_id)
            game["minute"] = game["minute"] + 1 # Convert to ordered time i.e: minute = 80 - event occurred in 81st minute
                                   
            # Extract the relevant events from each dataframe

            # Goals
            own_goals = game[game["type"] == "Own Goal Against"].reset_index(drop = True)
            own_goals = own_goals[["minute", "player", "team", "play_pattern"]]

            for index, row in own_goals.iterrows():           
            # Build a nested dictionary of all games
                dictionary[f"{match_date}: {home_team} vs. {away_team}"].update({f"Own Goal {index+1}":{
                "minute": row["minute"],
                "player": row["player"],
                "own goal against": row["team"],
                "play_pattern": row["play_pattern"]}})
            # Shots
            ##########
            game_shots = game[game["type"] == "Shot"]
            passes = game[game["type"] == "Pass"]

            game_shots = game_shots[["minute", "shot_type", "player", "shot_outcome", "team", "play_pattern", "shot_body_part", "shot_technique", "shot_key_pass_id", "location", "under_pressure", "shot_end_location", "shot_first_time", "shot_statsbomb_xg"]]
            shots_passes = pd.merge(game_shots, passes, how = "left", left_on = "shot_key_pass_id", right_on = "id", suffixes=('_shot', '_pass'))

            shots_passes = shots_passes[["minute_shot", "shot_type_shot","player_shot", "shot_outcome_shot", "team_shot", "play_pattern_shot", "shot_body_part_shot", "shot_technique_shot", "location_shot","under_pressure_shot", "location_pass", "pass_body_part", "pass_cross", "pass_end_location", "pass_height", "pass_length", "pass_recipient", "pass_shot_assist", "pass_technique", "pass_type", "play_pattern_pass", "player_pass", "possession_team", "shot_end_location_shot", "shot_first_time_shot", "under_pressure_pass", "shot_statsbomb_xg_shot"]]
            shots_passes = shots_passes[shots_passes["shot_outcome_shot"].isin(["Goal", "Saved"])].reset_index(drop = True)
            ##########
                    
            ##########
            for index, row in shots_passes.iterrows():           
            # Build a nested dictionary of all games
                dictionary[f"{match_date}: {home_team} vs. {away_team}"].update({f"Shot {index+1}":{
                "minute": row["minute_shot"],
                "shot type": row["shot_type_shot"],
                "player": row["player_shot"],
                "shot outcome": row["shot_outcome_shot"],
                "team": row["team_shot"],
                "play pattern": row["play_pattern_shot"],
                "shot body part": row["shot_body_part_shot"],
                "shot technique": row["shot_technique_shot"],
                "shot location": row["location_shot"],
                "key pass location": row["location_pass"],
                "pass body part": row["pass_body_part"],
                "pass cross": row["pass_cross"],
                "pass end location": row["pass_end_location"],
                "pass height": row["pass_height"],
                "pass length": row["pass_length"],
                "pass recipient": row["pass_recipient"],
                "pass shot assist": row["pass_shot_assist"],
                "pass technique": row["pass_technique"],
                "pass type": row["pass_type"],
                "play pattern pass": row["play_pattern_pass"],
                "passing player": row["player_pass"],
                "shot end location": row["shot_end_location_shot"],
                "statsbomb xg of shot": row["shot_statsbomb_xg_shot"]
                }})
                
            substitutions = game[game["type"] == "Substitution"].reset_index(drop = True)
            substitutions = substitutions[["minute", "player", "substitution_outcome", "substitution_replacement", "team"]]

            for index, row in substitutions.iterrows():
                 dictionary[f"{match_date}: {home_team} vs. {away_team}"].update({f"Substitution {index+1}": {
                 "Minute": row["minute"],
                 "Player Out": row["player"],
                 "Player In": row["substitution_replacement"],
                 "Substitution Type": row["substitution_outcome"],
                 "Team": row["team"]}})

            keeper_events = game[game["type"] == 'Goal Keeper']
            keeper_events = keeper_events.loc[keeper_events["goalkeeper_type"].isin(["Shot Saved", "Goal Conceded"])][['goalkeeper_body_part', 'goalkeeper_end_location', 'goalkeeper_outcome','goalkeeper_position', 'goalkeeper_technique', 'goalkeeper_type','minute','team', 'player']].reset_index()

            for index, row in keeper_events.iterrows():
                dictionary[f"{match_date}: {home_team} vs. {away_team}"].update({f"Goalkeeper Event {index+1}": {
                "Minute": row["minute"],
                "Player": row["player"],                  
                "Type": row["goalkeeper_type"],
                "Outcome": row["goalkeeper_outcome"],
                "Bodypart": row["goalkeeper_body_part"],
                "Team": row["team"]}})
                
            if ("foul_committed_card" in game.columns) & ("bad_behaviour_card" in game.columns):
                cards = game[(game["foul_committed_card"].isin(["Yellow Card", "Red Card"])) | (game["bad_behaviour_card"].isin(["Yellow Card", "Red Card"]))][["minute", "player", "foul_committed_card", "bad_behaviour_card","team"]].reset_index()
            elif "foul_committed_card" in game.columns:
                cards = game[game["foul_committed_card"].isin(["Yellow Card", "Red Card"])][["minute", "player", "foul_committed_card", "team"]].reset_index()
            elif "bad_behaviour_card" in game.columns:
                cards = game[game["bad_behaviour_card"].isin(["Yellow Card", "Red Card"])][["minute", "player", "bad_behaviour_card", "team"]].reset_index()
            try: #https://www.w3schools.com/python/python_try_except.asp
                for index, row in cards.iterrows():
                    if "bad_behaviour_card" in row:
                        card = row["bad_behaviour_card"]
                    else:
                        card = row["foul_committed_card"]
                    dictionary[f"{match_date}: {home_team} vs. {away_team}"].update({f"Card {index+1}": {
                    "Minute": row["minute"],
                    "Player": row["player"], 
                    "Card": card,                        
                    "Team": row["team"]}})
            except:
                pass
                
            if 'foul_won_penalty' in game.columns:
                penalties_won = game[game["foul_won_penalty"] == True][["minute", "player", "related_events", "team"]][:]
                penalties_won["related_events"] = penalties_won["related_events"].to_string(index = False).lstrip("[").rstrip("]")
                penalties_conceded = game[game["foul_committed_penalty"] == True][["id", "minute", "player", "team"]]
                penalties = pd.merge(penalties_won, penalties_conceded, how = "left", left_on = "related_events", right_on = "id", suffixes = ("_awarded", "_conceded"))
                penalties = penalties[["minute_awarded", "player_awarded", "team_awarded", "player_conceded", "team_conceded"]].reset_index()
            
            try:
                for index, row in penalties.iterrows():
                    dictionary[f"{match_date}: {home_team} vs. {away_team}"].update({f"Penalty {index+1}": {
                    "Minute": row["minute_awarded"],
                    "Player Awarded": row["player_awarded"],                  
                    "Team Awarded": row["team_awarded"],
                    "Player Conceded": row["player_conceded"],
                    "Team Conceded": row["team_conceded"]}})
                    
            except:
                pass

In [60]:
for i in range(0, selected_matches.shape[0]):
    match_data_dict = {}
    match_events_dict = {}
    match_data_extract(selected_matches.iloc[[i]], match_data_dict)
    match_events_extract(selected_matches.iloc[[i]], match_events_dict)
    match_data_dict["Events"] = match_events_dict
    print(json.dumps(match_data_dict, indent = 4))


C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


{
    "2020-10-04:Manchester United vs. Brighton & Hove Albion WFC": {
        "match_id": 3775562,
        "match_date": "2020-10-04",
        "match_result": "Home Win",
        "full_time_score": "3: 0",
        "match_week": 3,
        "stadium": "Leigh Sports Village Stadium",
        "referee": "Elliot Swallow",
        "home_manager": "Casey Stoney",
        "away_manager": "Hope Patricia Powell",
        "home_team": "Manchester United",
        "away_team": "Brighton & Hove Albion WFC",
        "winning team": "Manchester United",
        "half_time_score_home": 1,
        "half_time_score_away": 0,
        "home_goal_times": [
            9,
            69,
            91
        ],
        "away_goal_times": [],
        "home_clearances": 12,
        "away_clearances": 12,
        "home_shots": 22,
        "away_shots": 5,
        "home_shot_on_target": 9,
        "away_shot_on_target": 2,
        "home_possession": 53.0,
        "away_possession": 47.0,
        "home_corner

In [23]:
# For each selected match - create the JSON data file then prompt ChatGPT with few-shot prompt
for i in range(0, selected_matches.shape[0]):
    match_data_dict = {}
    match_events_dict = {}
    match_data_extract(selected_matches.iloc[[i]], match_data_dict)
    match_events_extract(selected_matches.iloc[[i]], match_events_dict)
    match_data_dict["Events"] = match_events_dict
    json_match_data = json.dumps(match_data_dict, indent = 4)

    client = OpenAI()

    completion = client.chat.completions.create(
    model="gpt-4o",
    store=True,
    messages=[{"role": "system", "content": 
             """You are a sports journalist. Your task will be to write a match report on a womens' football match after reading two examples of a
             match report and reading the instructions on how to write the report. First, read the mens' match report delimited by the triple asterixes
             and use it as a guide for the style and structure of your report. Next, read the instructions on how to write your own report.
             
***Example 1: 
Evanilson scored his first AFC Bournemouth goal while Dango Ouattara and Antoine Semenyo were also on target as south-coast rivals Southampton were brushed aside in a routine 3-1 victory.
Brazilian forward Evanilson steered past ex-Cherries goalkeeper Aaron Ramsdale on 17 minutes at the Vitality Stadium on Monday thanks to Marcus Tavernier's quick thinking.
Ouattara was credited with Bournemouth's second and Semenyo clinched the third in a wonderful first-half performance with a fine solo effort.
Taylor Harwood-Bellis reduced the areas early in a second half in which Southampton, to their credit, showed some fighting spirit, but it could not repair the damage of a disappointing opening 45 minutes.
Bournemouth's first home win of the season moves them 11th in the Premier League standings, while Southampton are 19th with just a solitary point from six matches.
How the match unfolded 
The hosts caught Saints napping for the opener when Tavernier dinked a quick free-kick into the area and Evanilson expertly guided his volley into the bottom-left corner.
It sparked a period of dominance from the hosts and they doubled their lead when Lewis Cook's shot, from Semenyo's cutback, hit Ouattara and diverted beyond Ramsdale.
Semenyo then got in on the act himself, shimmying this way and that on the right of the penalty area before drilling a lovely, low finish into the bottom-left corner in the 39th minute.
Harwood-Bellis stooped low to nod in from Mateus Fernandes' deflected cross five minutes after the restart, having been brilliantly denied by Kepa Arrizabalaga just prior, to give Southampton hope.
Substitute Ben Brereton Diaz bent an effort wide of the right post and brilliant defending from Milos Kerkez denied Yukinari Sugawara a simple finish as Southampton fought in vain to create a nervy ending.
Bournemouth forwards' delight as Evanilson gets off the mark 
Bournemouth forked out a reported £40.2million to Porto for Evanilson in the summer transfer window as they sought a replacement for Dominic Solanke, who was sold to Tottenham Hotspur.
The 24-year-old failed to score in his first four Premier League appearances and missed a penalty as the Cherries fell to a 1-0 defeat to Chelsea earlier this month.
But there were no signs of a forward lacking in confidence when he brilliantly steered home after showing an almost telepathic understanding with Tavernier.
Andoni Iraola, watching from the stands as he served a touchline ban, will hope it can spark a run of form for Evanilson, who was also denied in a one-on-one in the second half.
Evanilson will certainly get chances if Ouattara and Semenyo can continue to show the sort of form they did here as Bournemouth earned a first home win over Southampton since 2016.
Both wingers were a constant menace down the flanks, with the former putting in a couple of excellent deliveries from the byline, while the latter was influential for the second before brilliantly lacing in his third league goal of the campaign.
They may no longer have Solanke but there is plenty of talent to admire in this Cherries front line.
Saints just can't get marching 
One of the pre-match narratives surrounded Ryan Fraser's first return to the Vitality Stadium since an acrimonious exit in 2020.
The Saints winger was loudly booed every time he went near the ball but he was denied by Kepa after Maxwel Cornet had already seen an effort deflected wide as part of a bright start from the visitors.
But soon enough chants of "Fraser, what's the score?" were ringing out from the terraces as Southampton - still winless since returning to the top flight - completely lost control of the match.
While Evanilson will rightly take credit for a fine opener, question marks will be asked of how the Southampton defence just completely lost concentration, too.
Theirs will not be the only defence to struggle against Semenyo and Ouattara this season, but familiar failings reared their head in a poor first-half performance.
A triple substitution at the break, which saw Brereton Diaz, Joe Aribo and Ross Stewart enter the fray, brought about a spirited second-half showing, which yielded Harwood-Bellis' goal.
In truth, though, the damage was done by a first-half horror show that will leave boss Russell Martin needing to go back to the drawing board.

Example 2: 
Brentford and West Ham United shared the points in a keenly-contested London derby as Bryan Mbeumo’s goal was cancelled out by Tomas Soucek in a 1-1 draw at the Gtech Community Stadium.
Mbeumo volleyed home from close range after 38 seconds to give Brentford the perfect start, but the hosts could not build on their advantage and West Ham equalised early in the second half.
Soucek’s goal for the visitors came after 54 minutes following some intricate play around the penalty area.
Both sides spent the rest of the second half searching for a winner, but one was not forthcoming.
The point leaves Brentford 12th in the Premier League table, while West Ham are two places below them in 14th.
How the match unfolded 
For the third Premier League match in a row, Brentford opened the scoring inside the first minute and became the first team to ever achieve that feat.
The Bees kept the ball alive inside the penalty area and Fabio Carvalho headed the ball across to Mbeumo to produce a volleyed finish into the roof of the net inside 38 seconds.
That was the only shot on target in the first half in a competitive and tight affair at the Gtech Community Stadium.
The visitors grabbed an equaliser nine minutes into the second half when Lucas Paqueta and Jarrod Bowen linked up on the edge of the penalty area, before Michail Antonio’s effort deflected kindly into the path of Soucek to sweep home.
Yehor Yarmoliuk’s effort forced Alphonse Areola into the only save of note in the second half as both defences stood firm.
Bees quick off the mark again 
It took Brentford just 38 seconds to strike and, after goals in the opening minute against Tottenham Hotspur and Manchester City in their recent Premier League fixtures, they are gaining a reputation as fast starters.
Mbeumo is becoming the main man at Brentford with a fifth goal in six matches and a 14th goal involvement in 15 London derby appearances in the Premier League.
Once again though, they were unable to get a second and allowed the visitors to settle into the game.
Mbeumo did head over Mikkel Damsgaard’s inviting delivery, but the hosts never looked like adding to their early goal.
Thomas Frank’s side also never looked in danger of conceding an equalising goal, but a slice of luck for the visitors drew the contest level.
Brentford’s propensity to drop points from losing positions will be a cause of concern for Frank, but they look strong at home and a tricky test for any team.
Soucek salvages a point for Hammers 
After a week in which West Ham lost at home to Chelsea and were knocked out of the EFL Cup by Liverpool, they got off to the worst possible start in west London.
After conceding early, the visitors did gain more composure within the match but found it difficult to break down a stubborn Brentford defence.
Mohammed Kudus’ close-range volley was blocked and Antonio dragged an effort wide, but the Hammers' pressure for an equaliser lacked cutting edge.
Julen Lopetegui though will have been pleased with their defensive display, which was much improved from last weekend’s Chelsea defeat.
Their control of the contest continued into the second half and a fortuitous break inside the penalty area allowed Soucek to equalise.
The Hammers remain unbeaten away from home in the Premier League despite a mixed start to the season.
Lopetegui’s side will host Ipswich next weekend before facing Spurs and Manchester United in consecutive matches.***
Instructions:           
             You will be provided with a JSON file (delimited by triple backticks) of data from a football match. 
             Your task is to write a 1000 word match report using only the data from the JSON file. The JSON file consists of two parts. The first part
             contains match statistics and the second part, labelled with the 'Events' key, contains match highlights.
             Use the following structure for the match report: 
             1. Write a short paragraph of 3 to 5 sentences which mentions the date, home team, away team, match result, full-time score, goals scored, 
             and the minute of each goal.
             2. Write a paragraph including a chronological summary of the match highlights taken from data in the 'Events' section of 
             the JSON file. Highlights are defined as events labelled as 'Own Goal' or 'Shot' where the shot outcome is 'Goal' or 'Saved', or the shot
             technique is not 'Normal'. Descriptions of a shot must include the corresponding 'minute', 'player', 'play pattern', 'shot body part', 'team', 
             'shot outcome', 'shot technique', 'passing player', 'pass type', 'pass height', and 'pass length'. If the shot outcome is 'Saved', 
             look for a Goalkeeper Event that matches the minute and include the 'Player', 'Bodypart','Outcome', and 'Team' value from that Goalkeeper 
             Event.
             3. Write a short paragraph including the events from the home team perspective. Mention players from the home team 
             who appear frequently and summarise their performance. Include events labelled as 'Substitution' for the home team. Descriptions of substitutions should 
             must include the 'minute', 'Player In' and 'Player Out'. Also include events labelled as 'Card' for the home team. Descriptions of cards must 
             mention the 'Minute', 'Player', 'Card', and 'Team'.
             4. Write a short paragraph including the events from the away team perspective. Mention players from the away team 
             who appear frequently and summarise their performance. Include events labelled as 'Substitution' for the away team. Descriptions of substitutions should 
             must include the 'minute', 'Player In' and 'Player Out'. Also include events labelled as 'Card' for the away team. Descriptions of cards 
             must mention the 'Minute', 'Player', 'Card', and 'Team'.
             5. Use the first part of the JSON file which includes statistics to create a table with a column of summary statistics for each team.
             The table must include each team name, the full-time score, home team half-time score, away team half-time score, home clearances,
             away clearances, home shots, away shots, home shots on target, away shots on target, home possession, away possession, home corners,
             away corners, home fouls committed, away fouls committed, home offsides, away offsides, home passes, away passes, home tackles, away tackles,
             home touches, away touches, home yellow cards, away yellow cards, home red cards, away red cards. 

             Use this template delimited by the triple asterixes to format the table and replace the values. Ensure the correct team names are included.
             ***
             <table>
                        <thead>
                            <tr>
            <p>[home team name]</p> </td> <td> <p>Match Statistics</p> </td> <td><p>[away team name]</p> 
                            </tr>
                        </thead>
                        <tbody class="matchCentreStatsContainer"><tr> <td> 
<p>value</p> </td> <td> <p>Full-time score</p> </td> <td> <p>value</p> </td> </tr><td> <p>value</p> </td> <td> <p>Half-time score</p> </td> <td> <p>value</p> </td> </tr> <td><p>value</p> </td> <td> <p>Clearances</p> </td> <td> <p>value</p> </td> </tr><tr> <td><p>value</p> </td> <td> <p>Shots</p> </td> <td> <p>value</p> </td> </tr><tr> <td><p>value</p> </td> <td> <p>Shots on Target</p> </td> <td> <p>value</p> </td> </tr><tr> <td><p>value</p> </td> <td> <p>Possession (%)</p> </td> <td> <p>value</p> </td> </tr><tr> <td><p>value</p> </td> <td> <p>Touches</p> </td> <td> <p>value</p> </td> </tr><tr> <td> <p >value</p> </td> <td> <p>Passes</p> </td> <td> <p>value</p> </td> </tr><tr> <td> <p>value</p> </td> <td> <p>Tackles</p> </td> <td><p >value</p> </td> </tr><tr> <td> <p>value</p> </td> <td> <p>Corners</p> </td> <td> <p >value</p> </td> </tr><tr> <td> <p>value</p> </td> <td> <p>Offsides</p> </td> <td> <p >value</p> </td> </tr><tr> <td><p>value</p> </td> <td> <p>Fouls Conceded</p> </td> <td> <p >value</p> </td> </tr><tr> <td><p>value</p> </td> <td> <p>Yellow Cards</p> </td> <td> <p >value</p> </td> </tr><tr><td><p>value</p> </td> <td> <p>Red Cards</p> </td> <td> <p >value</p> </td> </tr><tr> <td><p class="">6</p> </td> </tr></tbody>
                    </table>

             ***            
             Reference the JSON file to ensure all data are correct but write creatively. 
             Ensure the data in the table agrees with the data in the text.
             Ensure the match report gives a fluent chronological narrative of the match. 
             Write in the style of an experienced sports journalist and use the example of a match report you read to guide the style and structure of
             your report.
             Output your response in HTML format."""},
    {"role": "user", "content": """```"""f"{json_match_data}""""```"""}])
    article = completion.choices[0].message.content
    print(article)
# Write to text file
    file = open(f"Method 2_Article {i}.txt", "w")
    file.write(article)
    file.close()

C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


```html
<!DOCTYPE html>
<html>
<head>
    <title>Match Report: Arsenal WFC vs. Reading WFC</title>
</head>
<body>

<h1>Match Report: Arsenal WFC vs. Reading WFC</h1>

<p>The Meadow Park Stadium witnessed an exhilarating contest on September 6, 2020, as Arsenal WFC claimed a dominant 6-1 victory over Reading WFC. Goals for the home team were plentiful, with Kim Little opening the scoring in the 15th minute, followed by strikes from Vivianne Miedema (33', 78'), Jill Roord (40', 63', 81'), while Danielle Carter netted a late consolation goal for Reading in the 89th minute.</p>

<h2>How the match unfolded</h2>

<p>The action commenced with a saved header from Reading's Amalie Vevle Eikeland in the 10th minute, via a free-kick assist from Fara Williams. However, Arsenal's Kim Little swiftly responded in the 15th minute with a volley goal from Leah Williamson's throw-in, firmly placing the ball past Grace Moloney. This initiated Arsenal's commanding presence, as Vivianne Miedema doubled the 

C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


```html
<!DOCTYPE html>
<html>
<head>
    <title>Match Report: Reading WFC vs. Bristol City WFC</title>
    <style>
        table {
            width: 100%;
            border-collapse: collapse;
        }
        th, td {
            padding: 8px 12px;
            border: 1px solid #ccc;
            text-align: left;
        }
        th {
            background-color: #f4f4f4;
        }
    </style>
</head>
<body>
    <h2>Match Report: Reading WFC vs. Bristol City WFC</h2>
    <p>
        On December 6, 2020, Reading WFC and Bristol City WFC played out a tightly contested 1-1 draw at the Select Car Leasing Stadium. Bristol City struck first with Emma Bissell's goal in the 43rd minute, but Fara Williams leveled for Reading WFC in the 50th minute. Despite numerous chances, both teams settled for a share of the points as the match ended in a draw.
    </p>

    <h3>How the match unfolded</h3>
    <p>
        The match began with Reading WFC pressing hard. As early as the 2nd minute, Ang

C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


...

```html
<p>On January 24, 2021, Manchester United clashed with Birmingham City WFC at Leigh Sports Village Stadium. Under light clouds and brisk conditions, the home team seized a commanding 2-0 victory in this Women's Super League fixture. The goals were clinched in quick succession as the second half dawned, with Leah Galton breaking the deadlock in the 46th minute, followed by Ella Toone striking decisively in the 81st minute. Birmingham struggled to pose any threat, going goalless despite their efforts.</p>

<p>The match began with Manchester United establishing their dominance early on. In the 12th minute, Ella Toone tested Birmingham's defense with a right-footed shot following a precise ground pass from Jackie Groenen after a throw-in play; however, her attempt was thwarted by Birmingham’s keeper Hannah Hampton, who executed a successful two-handed save. Leah Galton added to the pressure with a left-footed shot in the 17th minute during regular play, orchestrated by a low pass f

C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


<table>
    <thead>
        <tr>
            <th>Aston Villa</th>
            <th>Match Statistics</th>
            <th>West Ham United LFC</th>
        </tr>
    </thead>
    <tbody class="matchCentreStatsContainer">
        <tr>
            <td>0</td>
            <td>Full-time score</td>
            <td>0</td>
        </tr>
        <tr>
            <td>0</td>
            <td>Half-time score</td>
            <td>0</td>
        </tr>
        <tr>
            <td>31</td>
            <td>Clearances</td>
            <td>15</td>
        </tr>
        <tr>
            <td>7</td>
            <td>Shots</td>
            <td>26</td>
        </tr>
        <tr>
            <td>4</td>
            <td>Shots on Target</td>
            <td>5</td>
        </tr>
        <tr>
            <td>36.0</td>
            <td>Possession (%)</td>
            <td>64.0</td>
        </tr>
        <tr>
            <td>473</td>
            <td>Touches</td>
            <td>669</td>
        </tr>
        <tr>
          

C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
C:\Users\barry\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Match Report</title>
</head>
<body>
    <h1>Manchester United Triumph Over Brighton & Hove Albion with a Decisive 3-0 Victory</h1>
    <p>On October 4, 2020, at the Leigh Sports Village Stadium, Manchester United took on Brighton & Hove Albion WFC and emerged victorious with a comprehensive 3-0 win. The goals for Manchester United were scored by Ella Toone in the 9th minute, Alessia Russo in the 69th minute, and Jane Ross in an injury-time strike in the 91st minute, sealing a commanding home win.</p>
    
    <h2>How the Match Unfolded</h2>
    <p>The match started energetically with Manchester United awarded a penalty in the 8th minute, after Alessia Russo was fouled by Maya Le Tissier, who received a yellow card. Ella Toone confidently converted the penalty with her ri